In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv(r'../../results/computed-fuel-rates-oct2021-sep2022-test-10222023.csv', delimiter=',', skiprows=0, low_memory=False)

In [3]:
df.columns

Index(['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0', 'ServiceDateTime', 'DateKey', 'CalendarDate', 'Year',
       'Month', 'OpKey', 'Operator', 'RtKey', 'RtKey2', 'Route', 'RouteName',
       'TripKey', 'Trip', 'StopKey', 'Stop', 'StopName', 'Vehicle',
       'VehicleType', 'SeatedCap', 'TotalCap', 'Lat', 'Lon', 'Boards',
       'Alights', 'Onboard', 'Bike', 'Daytype', 'Hour', 'Minute', 'Second',
       'Date', 'Powertrain', 'VehicleModel', 'Vehicle_mass',
       'time_delta_in_seconds', 'dist', 'Speed', 'Acceleration', 'elevation',
       'grade', 'Energy'],
      dtype='object')

In [24]:
# Step 1: Identifying the Busiest Day
busiest_day_count = df.groupby('Date')['TripKey'].nunique()
busiest_day = busiest_day_count.idxmax()

# Step 2: Filtering Data for the Busiest Day
df_busiest_day = df[df['Date'] == busiest_day]

# Step 3: Find unique Routes
unique_routes = df_busiest_day['Route'].unique()

# Step 4: Create new dataframe

# Initialize a list to hold the data for the new DataFrame
new_df_data = []

# Iterating through each unique route
for route in unique_routes:
    # Filter the DataFrame for the current route
    df_route = df_busiest_day[df_busiest_day['Route'] == route]

    # Find the mode Powertrain for the unique vehicles for this route
    mode_powertrain = df_route.groupby('Vehicle')['Powertrain'].agg(pd.Series.mode).mode()[0]

    # Count the number of unique electric buses for this route
    num_electric_buses = df_route[df_route['Powertrain'] == 'electric']['Vehicle'].nunique()

    # Count the number of unique TripKey for this route
    num_unique_trips = df_route['TripKey'].nunique()

    # Append the data to the list
    new_df_data.append({'Route': route, 'Powertrain': mode_powertrain, 'num_electric_buses': num_electric_buses, 'num_unique_trips': num_unique_trips})

# Create the new DataFrame from the list
new_df = pd.DataFrame(new_df_data)

# Display the new DataFrame
new_df


,Route,Powertrain,num_electric_buses,num_unique_trips
0,R44,conventional,0,59
1,39E,conventional,0,22
2,9999,conventional,5,217
3,921,electric,1,28
4,804,conventional,2,10
5,802,conventional,1,9
6,B6,conventional,1,84
7,G2,conventional,1,129
8,G1,conventional,1,172
9,803,conventional,1,4


In [26]:
# Drop unknown route 9999
new_df = new_df.loc[new_df['Route']!='9999']
new_df

,Route,Powertrain,num_electric_buses,num_unique_trips
0,R44,conventional,0,59
1,39E,conventional,0,22
3,921,electric,1,28
4,804,conventional,2,10
5,802,conventional,1,9
6,B6,conventional,1,84
7,G2,conventional,1,129
8,G1,conventional,1,172
9,803,conventional,1,4
10,820,conventional,0,2


In [27]:
new_df.to_csv('../../results/route-bus-info-for-plotting.csv', index=False)